In [40]:
library(dplyr)
library(plotrix)
library(caret) # createDataPartition
library(nnet)
library(e1071)
library(randomForest)
library(party)
options(warn = -1)

randomForest 4.6-14
Type rfNews() to see new features/changes/bug fixes.

Attaching package: ‘randomForest’

The following object is masked from ‘package:ggplot2’:

    margin

The following object is masked from ‘package:dplyr’:

    combine

Loading required package: grid
Loading required package: mvtnorm
Loading required package: modeltools
Loading required package: stats4
Loading required package: strucchange
Loading required package: zoo

Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric

Loading required package: sandwich


In [6]:
mr <- read.csv("../Data/mushroom.csv", header= F)
head(mr)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
p,x,s,n,t,p,f,c,n,k,...,s,w,w,p,w,o,p,k,s,u
e,x,s,y,t,a,f,c,b,k,...,s,w,w,p,w,o,p,n,n,g
e,b,s,w,t,l,f,c,b,n,...,s,w,w,p,w,o,p,n,n,m
p,x,y,w,t,p,f,c,n,n,...,s,w,w,p,w,o,p,k,s,u
e,x,s,g,f,n,f,w,b,k,...,s,w,w,p,w,o,e,n,a,g
e,x,y,y,t,a,f,c,b,n,...,s,w,w,p,w,o,p,k,n,g


In [7]:
# 문자를 숫자로 변환
# as.numeric(mr$V2)

for(i in 2:23){
    mr[,i] <- as.numeric(mr[,i])
}

head(mr)

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23
p,6,3,5,2,7,2,1,2,5,...,3,8,8,1,3,2,5,3,4,6
e,6,3,10,2,1,2,1,1,5,...,3,8,8,1,3,2,5,4,3,2
e,1,3,9,2,4,2,1,1,6,...,3,8,8,1,3,2,5,4,3,4
p,6,4,9,2,7,2,1,2,6,...,3,8,8,1,3,2,5,3,4,6
e,6,3,4,1,6,2,2,1,5,...,3,8,8,1,3,2,1,4,1,2
e,6,4,10,2,1,2,1,1,6,...,3,8,8,1,3,2,5,3,3,2


In [9]:
unique(mr$V2) 
# V2에 대한 컬럼이 6개 필요 

[1] 6 1 5 3 4 2

In [12]:
# 각 컬럼별 DATA 확인
# table(mr$V2)

for(i in 2:23){
    cat('Column Name : ', colnames(mr)[i] , '\n')
    print(table(mr[,i]))
    cat('---------------------\n')
}

Column Name :  V2 

   1    2    3    4    5    6 
 452    4 3152  828   32 3656 
---------------------
Column Name :  V3 

   1    2    3    4 
2320    4 2556 3244 
---------------------
Column Name :  V4 

   1    2    3    4    5    6    7    8    9   10 
 168   44 1500 1840 2284  144   16   16 1040 1072 
---------------------
Column Name :  V5 

   1    2 
4748 3376 
---------------------
Column Name :  V6 

   1    2    3    4    5    6    7    8    9 
 400  192 2160  400   36 3528  256  576  576 
---------------------
Column Name :  V7 

   1    2 
 210 7914 
---------------------
Column Name :  V8 

   1    2 
6812 1312 
---------------------
Column Name :  V9 

   1    2 
5612 2512 
---------------------
Column Name :  V10 

   1    2    3    4    5    6    7    8    9   10   11   12 
1728   96  752  732  408 1048   64 1492   24  492 1202   86 
---------------------
Column Name :  V11 

   1    2 
3516 4608 
---------------------
Column Name :  V12 

   1    2    3    4    5 
2

In [16]:
# 작업 수행 후의 컬럼 갯수 파악
num <- c()

for(i in 2:23){
    num = append(num ,length(unique(mr[,i])))
}
num
sum (num) # 컬럼이 117개가 생성된다

[1]  6  4 10  2  9  2  2  2 12  2  5  4  4  9  9  1  4  3  5  9  6  7

[1] 117

In [24]:
# 별도의 DataFrame 생성
mr2 <- data.frame(V1 = mr$V1)
head(mr2)

# sort(unique(mr$V2))
for(i in 2:23 ){
    for(j in sort(unique(mr[,i]))){
        mr2 = cbind(mr2, mr[,i] == j)
    }
}

head(mr2)

ncol(mr2) # 118개 , V1컬럼 1개 + 23개 컬럼 원핫인코딩 처리 117개

V1
p
e
e
p
e
e


V1,"mr[, i] == j","mr[, i] == j.1","mr[, i] == j.2","mr[, i] == j.3","mr[, i] == j.4","mr[, i] == j.5","mr[, i] == j.6","mr[, i] == j.7","mr[, i] == j.8",...,"mr[, i] == j","mr[, i] == j.1","mr[, i] == j.2","mr[, i] == j.3","mr[, i] == j.4","mr[, i] == j.5","mr[, i] == j.6","mr[, i] == j.7","mr[, i] == j.8","mr[, i] == j.9"
p,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE
e,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
p,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,...,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE


[1] 118

In [25]:
str(mr2)

'data.frame':	8124 obs. of  118 variables:
 $ V1          : Factor w/ 2 levels "e","p": 2 1 1 2 1 1 1 1 2 1 ...
 $ mr[, i] == j: logi  FALSE FALSE TRUE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  TRUE TRUE FALSE TRUE TRUE TRUE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  TRUE TRUE TRUE FALSE TRUE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE TRUE FALSE TRUE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j: logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ mr[, i] == j: lo

In [26]:
# 컬럼 갯수가 많아서 structure를 다 못 보는 경우 
# 100번 부터 끝까지 보기
str(mr2[,100:ncol(mr2)]) # : <- '슬라이스'

'data.frame':	8124 obs. of  19 variables:
 $ mr[, i] == j   : logi  FALSE TRUE TRUE FALSE TRUE FALSE ...
 $ mr[, i] == j.1 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.2 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.3 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.4 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.5 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.6 : logi  FALSE FALSE FALSE FALSE TRUE FALSE ...
 $ mr[, i] == j.7 : logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.8 : logi  FALSE TRUE TRUE FALSE FALSE TRUE ...
 $ mr[, i] == j.9 : logi  TRUE FALSE FALSE TRUE FALSE FALSE ...
 $ mr[, i] == j.10: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.11: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.12: logi  FALSE FALSE FALSE FALSE FALSE FALSE ...
 $ mr[, i] == j.13: logi  FALSE TRUE FALSE FALSE TRUE TRUE ...
 $ mr[, i] == j.14: logi  FALSE FALSE FALSE FA

In [ ]:
# 정렬
library(dplyr)

In [32]:
# 정렬하기
# arrange(mr2, V1) # 컬럼 이름이 중복이라 정렬 할 수 없다. 컬럼이름 변경 필요
colnames(mr2) <- paste("V", 1:118 , sep = '')
head(mr2) 
# 단점 어디서부터 어디까지가 이전 컬럼 1개인지 모른다

V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V109,V110,V111,V112,V113,V114,V115,V116,V117,V118
p,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE
e,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
p,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,...,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE


### column이름과 one-hot encoding을 한번에 처리하기

In [34]:
mr3 <- data.frame(V1 = mr$V1)
# head(mr3)

count = 2
for(i in 2:23){
    for(j in sort(unique(mr[,i]))){
        mr3 = cbind(mr3 , mr[,i] == j)
        colnames(mr3)[count] <- paste("v",i,"_",j,sep="")
        count = count + 1
    }
}
head(mr3)

V1,v2_1,v2_2,v2_3,v2_4,v2_5,v2_6,v3_1,v3_2,v3_3,...,v22_4,v22_5,v22_6,v23_1,v23_2,v23_3,v23_4,v23_5,v23_6,v23_7
p,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE
e,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE
p,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,...,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,TRUE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE
e,FALSE,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,...,FALSE,FALSE,FALSE,FALSE,TRUE,FALSE,FALSE,FALSE,FALSE,FALSE


In [36]:
# 정렬하기
mr3 <- arrange(mr3 , V1)

In [38]:
# Sampling
set.seed(1234)
samp <- createDataPartition(mr3$V1, p=0.7, list=F)

# 학습용
data.train <- mr3[samp,]
# 검증용
data.test <- mr3[-samp,]

# 문제 및 정답 만들기
x <- subset(data.test, select = -V1)
y <- data.test$V1

In [44]:
# 랜덤포레스트
# 학습시키기 
model.rf <- randomForest(V1 ~ . , data = data.train , ntree = 100 )
# 검증하기 
pred <- predict(model.rf , x)
# 예측력과 오분류포
mean(pred == y)
table(pred , y) 

# 인공신경망
model.nnet <- nnet(V1 ~ . , data=data.train, size = 3)
# 검증
pred <- predict(model.nnet, x, type='class')
# 예측력과 오분류포
mean(pred == y)
table(pred , y)

# 의사결정나무
model.ctree <- ctree(V1 ~ . , data=data.train )
# 검증
pred <- predict(model.ctree, x)
# 예측력과 오분류포
mean(pred == y)
table(pred , y)

# SVM
model.svm = svm(V1 ~ . , data=data.train )
# 검증
pred <- predict(model.svm, x)
# 예측력과 오분류포
mean(pred == y)
table(pred , y)

# 로지스틱
model.mt <- multinom(V1 ~ . , data=data.train )
# 검증하기
pred <- predict(model.mt , x)
# 예측력과 오분류포
mean(pred == y)
as.matrix(table(pred , y)) 

[1] 1

    y
pred    e    p
   e 1262    0
   p    0 1174

# weights:  358
initial  value 4147.219689 
iter  10 value 713.527130
iter  20 value 279.383685
iter  30 value 200.048232
iter  40 value 195.820932
iter  50 value 195.790551
iter  60 value 195.701158
iter  70 value 193.442040
iter  80 value 193.362524
iter  90 value 192.607379
iter 100 value 188.588233
final  value 188.588233 
stopped after 100 iterations


[1] 0.9954844

    y
pred    e    p
   e 1251    0
   p   11 1174

...

[1] 0.9975369

    y
pred    e    p
   e 1262    6
   p    0 1168

# weights:  119 (118 variable)
initial  value 3942.621163 
iter  10 value 119.564501
iter  20 value 6.258442
iter  30 value 0.648195
iter  40 value 0.353389
iter  50 value 0.188963
iter  60 value 0.123492
iter  70 value 0.072709
iter  80 value 0.042991
iter  90 value 0.028542
iter 100 value 0.006974
final  value 0.006974 
stopped after 100 iterations


[1] 1

    y
pred    e    p
   e 1262    0
   p    0 1174